Attemp to validate the training test

In [1]:
%cd /content
!git clone https://github.com/ClupusVT/AI_machine_learning.git


/content
Cloning into 'AI_machine_learning'...
remote: Enumerating objects: 462, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 462 (delta 8), reused 27 (delta 4), pack-reused 423 (from 1)
Receiving objects: 100% (462/462), 85.51 MiB | 10.44 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [6]:
%cd /content
!pip install -q roboflow
!pip install -q unzip
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
Cloning into 'ultralytics'...
remote: Enumerating objects: 41838, done.
remote: Counting objects: 100% (576/576), done.
remote: Compressing objects: 100% (370/370), done.
remote: Total 41838 (delta 352), reused 370 (delta 201), pack-reused 41262 (from 1)
Receiving objects: 100% (41838/41838), 31.45 MiB | 20.79 MiB/s, done.
Resolving deltas: 100% (30993/30993), done.
/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.9

In [13]:
%cd /content
!curl -L "https://universe.roboflow.com/ds/I0Xj0fADqQ?key=IEi1HcEtIZ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   903  100   903    0     0   3622      0 --:--:-- --:--:-- --:--:--  3626
100  145M  100  145M    0     0  36.9M      0  0:00:03  0:00:03 --:--:-- 48.0M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/40cd38_7_6_png.rf.68ef7fcd663cdf0f5b96bacdbcd94e07.jpg  
 extracting: test/images/42ba34_1_5_png.rf.4337fde8fbe3640cf4351fb41ac9c3ca.jpg  
 extracting: test/images/42ba34_5_5_png.rf.ceb556bd4c92d41496ae1d8c56600c4a.jpg  
 extracting: test/images/4b770a_1_4_png.rf.5a45b3b841a06de414ceb802e34c136f.jpg  
 extracting: test/images/4b770a_3_6_png.rf.d0d9403f2f73ca0da7a6a1373c02b749.jpg  
 extracting: test/images/573e61_1_9_png.rf.e82af77d907cdc12dccbd6857f53b9b1.jpg  
 

In [ ]:
%cd /content
!wget http://14.225.204.171:9998/training_3_70_epoch_20.pt

/content
--2024-09-20 10:03:10--  http://14.225.204.171:9998/training_3_70_epoch_20.pt
Connecting to 14.225.204.171:9998... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6262799 (6.0M) [application/octet-stream]
Saving to: ‘training_3_70_epoch_20.pt’

training_3_70_epoch 100%[===================>]   5.97M   115KB/s    in 51s     

2024-09-20 10:04:02 (121 KB/s) - ‘training_3_70_epoch_20.pt’ saved [6262799/6262799]



In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging
from google.colab.patches import cv2_imshow

# Define paths
model_path = '/content/training_3_70_epoch_20.pt'  # Path to your trained model
input_folder = '/content/test/images'           # Path to the folder containing images
label_folder = '/content/test/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/1st_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/1st_model_final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '0': 'ball',
                '1': 'goalkeeper',
                '2': 'football-players',
                '3': 'referee',
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")

In [3]:
%cd /content
!kaggle datasets download -d mnhthinguyn/player-detection-dataset


/content
Dataset URL: https://www.kaggle.com/datasets/mnhthinguyn/player-detection-dataset
License(s): other
  0% 0.00/908k [00:00<?, ?B/s]
100% 908k/908k [00:00<00:00, 83.3MB/s]


In [4]:
!pip install unzip
!unzip /content/player-detection-dataset.zip

  Preparing metadata (setup.py) ... done
  Created wheel for unzip: filename=unzip-1.0.0-py3-none-any.whl size=1280 sha256=9d8b5a0c6f0341524e77d0cf0e8f7335b85d575d8f0b5534ae20caa179824682
  Stored in directory: /root/.cache/pip/wheels/80/dc/7a/f8af45bc239e7933509183f038ea8d46f3610aab82b35369f4
Successfully built unzip
Archive:  /content/player-detection-dataset.zip
  inflating: Juve_1.mp4              


In [12]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('/content/AI_machine_learning/Player_Detection/training_3_70_epoch_20.pt')  # Replace with your model path

# Load the video
video_path = '/content/Juve_1.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Output video setup
output_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference on the frame
    results = model(frame)

    # Get the detection results
    detections = results[0].boxes.xyxy.cpu().numpy()  # bounding boxes
    confidences = results[0].boxes.conf.cpu().numpy()  # confidence scores
    class_ids = results[0].boxes.cls.cpu().numpy()  # class IDs

    # Filter out detections for players (assuming class IDs for players are known)
    #player_detections = detections[class_ids == YOUR_PLAYER_CLASS_ID]

    # Count the number of players detected
    player_count = len(detections)

    # Draw bounding boxes and player count on the frame
    for (x1, y1, x2, y2) in detections:
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Display player count
    cv2.putText(frame, f'Players: {player_count}', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the frame to the output video
    out.write(frame)

    frame_number += 1
    print(f'Processed frame {frame_number}/{frame_count}, Players detected: {player_count}')

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Output video saved at: {output_path}')



0: 384x640 1 football-players, 145.9ms
Speed: 6.9ms preprocess, 145.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 1/200, Players detected: 1

0: 384x640 (no detections), 146.9ms
Speed: 4.8ms preprocess, 146.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 2/200, Players detected: 0

0: 384x640 (no detections), 141.2ms
Speed: 5.9ms preprocess, 141.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 3/200, Players detected: 0

0: 384x640 (no detections), 159.5ms
Speed: 6.9ms preprocess, 159.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 4/200, Players detected: 0

0: 384x640 (no detections), 129.3ms
Speed: 5.0ms preprocess, 129.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processed frame 5/200, Players detected: 0

0: 384x640 (no detections), 127.3ms
Speed: 4.1ms preprocess, 127.3ms inference, 0.5ms postprocess per imag